This is a small tutorial that presents the different functionalities of the MusicTreequence library.

First we have to load the library. The following lines of code load all functions and classes from the library into the global namespace (so we can use them without having to type 'MusicTrequence' all the time).

In [1]:
from MusicTreequence import *
%load_ext autoreload
%autoreload 2

# A Simple Melody

The simplest way to produce a couple of tones is to use the parse function of the Event class.

In [2]:
with write_song(file='song.rb', print_to_std_out=True):
    fifth = Event.parse("g g g eb")
    fifth.write()

# main function
def song
  play 55, attack: 0.01, decay: 1.0, sustain: 0.1, release: 0.1, amp: 1.0
    sleep 1.0
  play 55, attack: 0.01, decay: 1.0, sustain: 0.1, release: 0.1, amp: 1.0
    sleep 1.0
  play 55, attack: 0.01, decay: 1.0, sustain: 0.1, release: 0.1, amp: 1.0
    sleep 1.0
  play 51, attack: 0.01, decay: 1.0, sustain: 0.1, release: 0.1, amp: 1.0
    sleep 1.0
end

# function for testing infinite loops
def loop_test(key)
  loops = [
  ].to_set
  return loops.include?(key)
end

# symbols




This is what has been writen to the file 'song.rb'. Exectue the file 'main.rb' in Sonic Pi to listen to it. The main function called 'song' is called by Sonic Pi. The second and third part (the stuff with the loops and symbols) will become clear later. It has no function at the moment.

The first line of code specifies where everything should be written. The 'file' parameter may contain a file name and the 'print_to_std_out' parameter specifies whether the generated Ruby code should be printed to standard output.

The second line of code calls the 'parse' function of the Event class, which reads the given string and returns an event.

The third line of code writes the event as Ruby code.

### Be more verbose

We can be a bit more precise. This will also sound better.

In [3]:
with write_song(file='song.rb'):
    fifth = Event.parse("r/8 g./8 g./8 g./8 eb/2")
    fifth.write()

What happens behind the scenes is that the string is parsed as a sequence of events with specific properties. We could also directly specify this sequence.

In [4]:
with write_song(file='song.rb'):
    t = Tone(pitch='g', duration='1/8', staccato=True)
    Sequence([
        Rest(extent='1/8'),
        t,
        t,
        t,
        Tone(pitch='eb', duration='1/2')
    ]).write()

As you can see Rest and Tone are separate events, which are grouped in a Sequence event. If you look at the generated output you can see that it is identical to the one produced by the parse function. If you directly write out the Tone events you will find that the 'sleep' statements are missing. This is because a Tone on its own does not know what is happening before or after it. Always use one of these 'container' events to group elementary events.

# Elementary Events

There are four elementary event: Chord, Tone, Beat, and Rest. Actually Tone, which we already used, is only a specialization of Chord.

In [5]:
with write_song(file='song.rb'):
    Sequence([
        Chord(intervals=[0, 4, 7], base='eb', duration='1/4'),
        Beat(extent='1/4', sound='tab'),
        Beat(extent='1/4', sound='snare'),
        Beat(extent='1/4', sound='kick'),
        Beat(extent='1/4', sound='hh_c'),
        Beat(extent='1/4', sound='hh_o'),
        Rest(extent='1/4'),
        Beat(extent='1/4', sound='ride'),
    ]).write()

# Container Events

There are two container event. One is the Sequence event from above. The other one is the Parallel event, which does what its name suggests: It plays stuff simultaneously.

In [6]:
with write_song(file='song.rb'):
    Parallel([
        Event.parse("r/8 bb./8 c'./8 d'./8 bb/2"),
        Event.parse("r/8 g./8 g./8 g./8 eb/2")
    ]).write()

Additionally there is the Measure event that takes a list of events and fiddles around with their durations and amplitudes. It's a bit complicated but it can produce nice effects.

In [7]:
with write_song(file='song.rb'):
    tab = Beat(sound='tab')
    Sequence([
        Measure([tab] * 8, extent=4),
        Rest('1/2'),
        [Measure([tab, [tab, tab, tab], tab, [tab, tab, tab, tab]], extent=4)] * 2,
        Rest('1/2'),
        Measure(repack_list(list_to_repack=[tab] * 16, package_sizes=(2, 2, 2)), extent=4),
    ]).write()

# Special Events

### Transposed

### Symbol

### Loop

# Time Series Models

### Static Pitch Distributions

###### PitchDistribution

###### ScaleDistribution

###### PitchRange

### TimeSeriesProduct

### MarkovModel

### Inference